In [1]:
import pandas as pd
import numpy as np
import math
import re
import time
import glob
import os
import json

In [2]:
class Atom(object):
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):
        return f"x={self.x} - y={self.y} - z={self.z}"
    def __repr__(self):
        return self.__str__()
        
class Cal_Atom(object):
    def __init__(self, carbon, hidro):
        self.carbon = carbon
        self.hidro = hidro
    
    def cal_cos_alpha_c_h(self):
        V_ch = self.cal_distance_c_h()
        cos_alpha = None
        if V_ch != 0:
            cos_alpha = self.carbon.z/V_ch
        print('Z carbon: ', self.carbon.z)
        print('cos_alpha: ', cos_alpha)
        return cos_alpha

    def cal_distance_c_h(self):
        print('X carbon: ',self.carbon.x)
        print('Y carbon: ',self.carbon.y)
        print('Z carbon: ',self.carbon.z)
        print('X hidro: ',self.hidro.x)
        print('Y hidro: ',self.hidro.y)
        print('Z hidro: ',self.hidro.z)
        output = math.sqrt(pow((self.carbon.x-self.hidro.x), 2) + pow((self.carbon.y-self.hidro.y), 2) + pow((self.carbon.z-self.hidro.z), 2))
#         print('Distance C_H: ', output)
        return output

    def cal_S(self):
        cos_alpha = self.cal_cos_alpha_c_h()
        if cos_alpha:
            return 0.5*((3*pow(cos_alpha, 2)) - 1)
        return None
    
def get_index_carbon(index_file):
    index_dict = {}
    pattern = "\[(.*?)\]"
    with open(index_file) as index_file:
        lines = index_file.readlines()
        for line in lines:
            if "[" in line and line not in index_dict:
                key = re.search(pattern, line).group(1).strip()
                index_dict[key] = []
                continue
            if line.strip() != "":
                index_dict[key].append(int(line.strip()))
    return index_dict


class Lipid(object):
    pass

In [3]:
def main():
    output = []
    carbon_indices = get_index_carbon(index_file)     
    for carbon_name, index_list in carbon_indices.items():
        carbon_name = carbon_name.strip()
        print('*'*30)
        print(carbon_name)
        value_cal = 0
        for index_item in index_list:
            print('INdex:', index_item)
            print('*'*10)
            carbon_value = df.loc[[index_item]]
            hidro_value = df.loc[[index_item + 1]]
            cx = carbon_value['z'].values[0]
            cy = carbon_value['a'].values[0]
            cz = carbon_value['b'].values[0]
            carbon = Atom(cx, cy, cz)
            hx = hidro_value['z'].values[0]
            hy = hidro_value['a'].values[0]
            hz = hidro_value['b'].values[0]
            hidro = Atom(hx, hy, hz)
            calculate_atom = Cal_Atom(carbon, hidro)
            value_cal += abs(calculate_atom.cal_S())
        value_cal = value_cal / len(index_list)
        print(f'average value: {value_cal}')

In [4]:
lipid_dict = {
    'CHL1' : {
        'c3':0,
        'c17':23
    },
    'DPPC' : {
        'p':0,
        'c2':8,
        'c216':64,
        'c316':107
    },
    'POPC' : {
        'p':0,
        'c2':8,
        'c218':20,
        'c316':63
    },
    'PSM' : {
        'p':0,
        'c25':10,
        'c185':56,
        'c16f':104
     }
}

In [5]:
index_file_paths = glob.glob('index_folder/*.ndx')
index_file_paths

['index_folder\\C3_CHL1.ndx',
 'index_folder\\P_DPPC.ndx',
 'index_folder\\P_POPC.ndx',
 'index_folder\\P_PSM.ndx']

In [6]:
df = pd.read_csv('clean_coor_1.gro')
df.index += 1
df

,lipid,atom,index,x,y,z,a,b,c
1,1CER18,C1S,1.0,3.138,2.531,8.244,0.2793,-0.4823,0.4629
2,1CER18,H1S,2.0,3.218,2.464,8.281,0.0112,-0.1649,1.6452
3,1CER18,H1T,3.0,3.038,2.498,8.279,0.6613,-3.4690,-0.9641
4,1CER18,NF,4.0,3.099,2.389,8.046,-0.1134,-0.8263,0.4335
5,1CER18,HNF,5.0,3.163,2.332,7.996,-1.2825,-0.5325,-1.4804
...,...,...,...,...,...,...,...,...,...
73108,598PSM,H15G,73108.0,0.558,0.239,6.290,0.4743,0.5631,-1.6617
73109,598PSM,C16F,73109.0,0.353,0.219,6.373,0.8269,-0.3447,0.8218
73110,598PSM,H16F,73110.0,0.361,0.266,6.473,-0.1706,-1.3416,1.3990
73111,598PSM,H16G,73111.0,0.253,0.228,6.328,0.6390,-1.6915,0.9148


In [8]:
 for file_path in index_file_paths:
    _, filename = os.path.split(file_path)
    print('==> file name: ',filename)
    lipid_name = filename.split('.')[0].split('_')[-1]
    print('==> Lipid name: ', lipid_name)
    with open(file_path) as file:
        index_file_content = file.read()
        index_file_content = index_file_content.strip().replace('\n', ' ').split(' ')
        lipid_atom_offset = lipid_dict[lipid_name]
        print(lipid_atom_offset)
        for p_index in index_file_content:
            for key, val in lipid_atom_offset.items():
                lipid_object = Lipid()
                atom_name = key
                atom_index = val+int(float(p_index))
                print(f"Atom: {atom_name} ---> index: {atom_index}")
                atom_x = df.loc[int(atom_index), 'x']
                atom_y = df.loc[int(atom_index), 'y']
                atom_z = df.loc[int(atom_index), 'z']
                atom = Atom(atom_x, atom_y, atom_z)
                print(atom)
                setattr(lipid_object, atom_name, atom)
                vector = []
            print('*'*30)
                

==> file name:  C3_CHL1.ndx
==> Lipid name:  CHL1
{'c3': 0, 'c17': 23}


ValueError: could not convert string to float: '['

In [ ]:
def grid_surface_area(x_vals, y_vals, zgrid):
    """Compute the surface area across a regular 2d grid.
    Args:
        x_vals (np.array): The bin labels along the x-axis of the gridded data.
        y_vals (np.arrray): The bin labels along the y-axis of the gridded data.
        zgrid (np.array): The 2d grid of bin values.

    Returns:
        float: Returns a the surface area estimate.
    """
    nxb = len(x_vals)
    nyb = len(y_vals)
    x_incr = x_vals[1]-x_vals[0]
    y_incr = y_vals[1]-y_vals[0]
    # print("x_incr {} y_incr {}".format(x_incr, y_incr))
    [sy, sx] = np.gradient(zgrid, y_incr, x_incr)
    #now get curvatures
    sa = 0.0
    for ix in range(nxb):
        for iy in range(nyb):
            #upper
            sx_c = sx[ix, iy]
            sy_c = sy[ix, iy]
            sx_v = np.array([1.0, 0.0, sx_c])
            sy_v = np.array([0.0, 1.0, sy_c])
            cross = np.cross(sx_v, sy_v)
            dA = np.sqrt(np.dot(cross, cross))*x_incr*y_incr
            sa += dA
    return sa